# 목표 최적화

이 노트북에서는 Amazon Personalize 목표 최적화의 사용 사례를 보여줍니다. 추천에 영향을 미치는 다른 여러 가지 요인이 있는 경우가 많으며, 목표 최적화를 사용하여 모델을 구동하기 위한 추가 데이터를 제공할 수 있습니다.

이 예에서는 Video On Demand(VOD) 스트리밍 구독 서비스에 대한 추천을 제공한다고 가정합니다. 이 서비스는 사용자가 동영상 카탈로그에 무제한으로 액세스할 수 있도록 허용하지만, 카탈로그에서 제공되는 타이틀은 콘텐츠 소유자와의 계약에 따라 라이선스 조건이 다릅니다. 이 스트리밍 서비스는 라이선스 비용이 낮은 콘텐츠에 가중치를 부여하는 추천을 제공함으로써, 고객의 만족도를 높이면서도 라이선스 비용을 낮출 수 있습니다.

다시 말하지만, 데이터는 [MovieLens](https://movielens.org/) 프로젝트에서 가져오며, 아래 셀에서 기다리는 시간 동안 웹 검색을 통해 데이터와 그 잠재적 사용 사례를 알아볼 수 있습니다. MovieLens 데이터를 추가적인 로열티 필드로 보완하고, 데이터 세트에 로열티 비용을 제공하는 데이터를 생성합니다. 

## 이 노트북을 사용하는 방법

코드는 아래와 같이 셀로 구분됩니다. 이 페이지 맨 위에 삼각형 모양의 `Run` 버튼이 있습니다. 이 버튼을 클릭하여 각 셀을 실행하고 다음 셀로 이동하거나, 셀에 있는 동안 `Shift` + `Enter`를 눌러 셀을 실행한 후 다음 셀로 이동할 수 있습니다.

셀이 실행되면, 셀이 실행되는 동안에는 옆의 라인에 `*`가 표시되며, 셀 내의 모든 코드 실행을 완료하고 나면 이 기호가 실행을 완료한 마지막 셀을 나타내는 숫자로 업데이트됩니다.


아래 지침에 따라 셀을 실행하여 Amazon Personalize 목표 최적화를 시작하세요.

## 가져오기 

Python은 다양한 라이브러리 컬렉션과 함께 제공되며, 핵심 데이터 과학 도구로서 설치된 [boto3](https://aws.amazon.com/sdk-for-python/)(python용 AWS SDK), [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) 등을 라이브러리와 함께 가져와야 합니다.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from botocore.exceptions import ClientError

!conda install -y -c conda-forge unzip

pd.options.mode.chained_assignment = None

그런 다음 사용자 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 검증해야 합니다. 아래 코드 줄은 이 작업을 수행합니다.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

## 데이터 구성

데이터는 Amazon S3을 통해 Amazon Personalize로 가져옵니다. 아래에서는 파일을 S3에 업로드하기 전에 로컬에서 파일을 작성하는 데 사용할 파일 이름을 지정합니다.


In [ ]:
filename = "movie-lens-100k.csv"
items_filename = "movie-lens-items.csv"

### 훈련 데이터 다운로드, 준비 및 업로드

현재 MovieLens 데이터가 아직 로컬로 로드되지 않았습니다. 아래 코드 줄을 실행하여 최신 복사본을 다운로드하고 간단히 검사합니다.

#### 데이터 세트 다운로드 및 탐색

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### 데이터 준비 및 업로드

아래의 코드는 MovieLens 다운로드에서 동영상 정보를 로드합니다. 타이틀, 릴리스 날짜, IMDB 링크, 그리고 타이틀에 적용되는 장르를 캡처할 열 목록에 대한 정보를 제공합니다.

이 워크숍에서는 가공의 로열티를 생성하므로, 로열티 필드는 없으며 다음 단계에서 해당 작업을 수행합니다.








In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|',encoding='latin-1', names=['ITEM_ID', 'TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL', 'MISC', 'ACTION_GENRE', 'ADVENTURE_GENRE', 'ANIMATION_GENRE', 'CHILDRENS_GENRE','COMEDY_GENRE', 'CRIME_GENRE', 'DOCUMENTARY_GENRE','DRAMA_GENRE','FANTASY_GENRE', 'FILMNOIR_GENRE','HORROR_GENRE', 'MUSICAL_GENRE','MYSTERY_GENRE', 'ROMANCE_GENRE', 'SCIFI_GENRE', 'THRILLER_GENRE', 'WAR_GENRE', 'WESTERN_GENRE'     ])
items

#### 로열티 데이터 및 장르 추가
ROYALTY 필드에 0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10으로 균등하게 분포된 값을 할당합니다. 그러면 대부분의 영화는 로열티가 상대적으로 낮거나 0이 되고, 소수의 영화만 로열티가 높게 됩니다.

아래의 막대 차트에서 각 로열티 값이 타이틀에 균등하게 분포되어 있는 것을 알 수 있습니다.

위에서 MovieLens GERE 필드 결과는 타이틀이 해당 장르에 속하는지 여부를 나타내는 각 열이 있는 범주형 필드이지만, 개인화에 로드된 항목에는 배열이 사용될 수 있습니다. 일부 Pandas는 모든 장르를 나열하는 파이프 문자로 구분된 열에 장르를 취합하는 코드를 작성합니다.



In [ ]:
pd.set_option('display.max_rows', 10)

royaltyvalues = [0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.05, 0.10]
items.loc[:,'ROYALTY'] = items['ITEM_ID'].map(lambda x: royaltyvalues[x%8])

items.loc[:,'GENRE']='' 

for col_name in items.columns:
    if col_name.endswith('_GENRE'):
        items.loc[items[col_name]==1,'GENRE']= items['GENRE']+'|'+ col_name[:-6]

items = items[['ITEM_ID', 'TITLE','ROYALTY', 'GENRE']]
items.loc[:,'GENRE'] = items['GENRE'].str[1:]


items.loc[:,'ROYALTY'].value_counts().plot.bar()
items.head(10)


#### 로열티 값을 음수로 조정합니다.

목표 최적화는 필드의 가장 높은 값으로 최적화합니다. 이 예에서는 스트리밍 서비스로 인해 발생하는 로열티 비용과 영화의 연관성을 균형 있게 조정하려고 합니다.

로열티가 가장 낮은 영화를 많이 시청하도록 하기 위해서는 로열티가 가장 낮은 영화에 가장 높은 값을 할당해야 합니다. 이 예에서는 로열티를 음수로 변환합니다. 이를 위해 로열티 수수료의 절대값에 -1을 곱합니다.

참고: 이 워크숍 환경에서 유용한 동일한 결과로 이 셀을 두 번 실행할 수 있도록 절대값이 사용되었습니다.

In [ ]:
items.loc[:,'ROYALTY'] = -1 * abs(items['ROYALTY'])
items

## S3 버킷 및 IAM 역할 구성 <a class="anchor" id="bucket_role"></a>
지금까지 이 Jupyter 노트북을 실행하는 인스턴스에 연결된 Amazon EBS 인스턴스에 데이터를 다운로드하고 조작하고 저장했습니다. 하지만 Amazon Personalize에는 데이터의 소스 역할을 하는 S3 버킷과 해당 버킷에 액세스하기 위한 IAM 역할이 필요합니다. 이 버킷과 역할을 설정해보겠습니다.

이 Amazon SageMaker 노트북의 기반이 되는 인스턴스에 저장된 메타데이터를 사용하여 해당 노트북이 운영되고 있는 리전을 확인합니다. Amazon SageMaker 외부에서 Jupyter 노트북을 사용하는 경우 아래의 문자열로 리전을 정의하면 됩니다. Amazon S3 버킷은 지금까지 생성한 Amazon Personalize 리소스와 동일한 리전에 있어야 합니다.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    notebook_data = json.load(notebook_info)
    resource_arn = notebook_data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Amazon S3 버킷 이름은 전역적으로 고유합니다. 고유한 버킷 이름을 만들기 위해 아래 코드는 AWS 계정 번호에 `personalize-objective-optimization-` 문자열을 추가합니다. 그런 다음 이전 셀에서 확인된 리전에 이 이름으로 버킷을 만듭니다.

In [ ]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-objective-optimization-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 항목 데이터의 CSV 파일을 업로드합니다. 

In [ ]:
items = items[['ITEM_ID', 'TITLE', 'GENRE', 'ROYALTY']]
items_dataset = items[['ITEM_ID','ROYALTY', 'GENRE']]

items_dataset.to_csv(items_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(items_filename).upload_file(items_filename)

### S3 버킷 정책 설정
Amazon Personalize가 S3 버킷의 콘텐츠를 읽을 수 있어야 합니다. 이를 허용하는 버킷 정책을 추가합니다.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

#### 영화 타이틀 항목 스키마 생성
두 개의 스키마가 필요합니다. 그중 하나는 영화 타이틀에 대한 스키마로 항목 유형이 되고. 다른 하나는 상호 작용의 구조를 정의합니다.

In [ ]:
schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ROYALTY",
            "type": "float"
        },        {
            "name": "GENRE",
            "type": [
                "null",
                "string"
              ],
            "categorical": True
        }
    ],
    "version": "1.0"
}

create_item_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-item-schema"+suffix,
    schema = json.dumps(schema)
)

item_schema_arn = create_item_schema_response['schemaArn']
print(json.dumps(create_item_schema_response, indent=2))

이 시점부터 [첫 번째 캠페인](https://github.com/aws-samples/amazon-personalize-samples/blob/master/getting_started/notebooks/1.Building_Your_First_Campaign.ipynb)과 동일한 프로세스를 수행하여 상호 작용을 로드합니다. 평점이 3점 이상인 상호 작용만 포함해야 하고, 시청자들이 좋아하지 않을 영화를 추천해서는 안 됩니다.

In [ ]:
data = data[data['RATING'] > 3]                # Keep only movies rated higher than 3 out of 5.
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

### 상호 작용 스키마 생성

Personalize가 데이터를 이해하기 위한 핵심 구성 요소는 아래에 정의된 스키마에서 얻어집니다. 이 구성은 Personalize 서비스에 CSV 파일을 통해 제공된 데이터를 수집하는 방법을 알려줍니다. 열과 유형은 위에서 만든 파일에 있는 내용과 일치합니다.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-objective-optmization-schema"+suffix,
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### 데이터 세트 그룹 생성 및 생성이 완료될 때까지 기다리기

Personalize에서 가장 큰 그룹은 데이터 세트 그룹으로, 데이터, 이벤트 트래커, 솔루션 및 캠페인을 격리합니다. 공통적인 데이터 컬렉션을 공유하는 것들끼리 한데 그룹화합니다. 원하는 경우 아래의 이름을 자유롭게 변경해도 됩니다.

#### 데이터 세트 그룹 생성

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-objective-optmization-demo-"+suffix
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### 데이터 세트 그룹이 활성 상태가 될 때까지 기다리기

아래의 항목에서 데이터 세트 그룹을 사용하려면 먼저 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### 데이터 세트 생성

그룹을 생성한 후에는 실제 데이터 세트를 만들어야 합니다. 이 예에서는 상호 작용 데이터의 데이터 세트를 하나 만들고 항목 데이터의 데이터 세트를 또 하나 만듭니다. 아래의 셀을 실행하여 데이터 세트를 만듭니다.

In [ ]:
def create_dataset(dataset_type, schema_arn, name):
    create_dataset_response = personalize.create_dataset(
        name = name,
        datasetType = dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = schema_arn
    )
    dataset_arn = create_dataset_response['datasetArn']

    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = dataset_arn
        )
        status = describe_dataset_response["dataset"]["status"]
        print("Dataset: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(10)
    return dataset_arn


In [ ]:
interaction_dataset_arn = create_dataset("INTERACTIONS", schema_arn, 'personalize-objective-optmization-interactions-'+suffix)


print('interaction_dataset_arn: ' + interaction_dataset_arn)


In [ ]:
item_dataset_arn = create_dataset("ITEMS", item_schema_arn, 'personalize-objective-optmization-items-'+suffix)
print('item_dataset_arn: ' + item_dataset_arn)

#### Personalize 역할 생성

또한 Amazon Personalize가 특정 작업을 실행할 수 있는 권한을 가지려면, AWS에서 역할을 수임할 수 있어야 합니다. 아래의 코드 줄은 이 권한을 부여합니다.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleDemo"+suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## 데이터 가져오기

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 모델 구축 사용 사례를 위해 S3에서 Amazon Personalize로 데이터를 로드하는 가져오기 작업을 실행합니다. 여러 가져오기를 로드하므로, 아래의 함수를 호출하여 가져오기 작업을 시작한 다음 완료 여부를 모니터링합니다.

#### 데이터 세트 가져오기 작업 생성

In [ ]:
def create_dataset_import_job(dataset_arn, dataLocation, name):
    create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = name,
        datasetArn = dataset_arn,
        dataSource = {
            "dataLocation": dataLocation
        },
        roleArn = role_arn
    )

    dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_dataset_import_job_response = personalize.describe_dataset_import_job(
            datasetImportJobArn = dataset_import_job_arn
        )
        status = describe_dataset_import_job_response["datasetImportJob"]['status']
        print("DatasetImportJob: {} {}".format(name, status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)
    return dataset_import_job_arn

#### 상호 작용 로드
아래의 가져오기 작업은 상호 작용 데이터 세트를 로드합니다.

In [ ]:
dataset_import_job_arn = create_dataset_import_job(interaction_dataset_arn, "s3://{}/{}".format(bucket_name, filename), "personalize-objective-optimization-interaction-"+suffix)
print('dataset_import_job_arn: ' + dataset_import_job_arn)

#### 항목 데이터 세트에 영화 타이틀을 로드합니다.

가져오기 작업이 완료되는 데 시간이 걸릴 수 있습니다. 아래에 활성 상태로 표시될 때까지 기다리세요.

In [ ]:
item_dataset_import_job_arn = create_dataset_import_job(item_dataset_arn, "s3://{}/{}".format(bucket_name, items_filename), "personalize-objective-optimization-item-"+suffix)
print('item_dataset_import_job_arn: ' + item_dataset_import_job_arn)

## 솔루션 및 버전 생성

Amazon Personalize에서는 훈련된 모델을 솔루션이라고 하며, 각 솔루션은 모델을 훈련할 때 주어진 데이터 볼륨과 관련된 여러 버전을 가질 수 있습니다.

먼저 지원되는 모든 레시피를 나열합니다. 레시피는 데이터를 아직 훈련하지 않은 알고리즘입니다. 목록을 작성한 후 하나를 선택하여 모델을 구축하는 데 사용할 수 있습니다.

### 레시피 선택

#### 사용자 개인화
[사용자 개인화](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html)(aws-user-personalization) 레시피는 모든 USER_PERSONALIZATION 추천 시나리오에 최적화된 레시피입니다. 이 레시피는 항목을 추천할 때 자동 항목 탐색을 사용합니다.

자동 탐색 기능을 통해 Amazon Personalize는 자동으로 여러 가지 항목 추천을 테스트하고, 사용자가 이러한 추천 항목과 상호 작용하는 방법을 학습하며, 더 나은 참여와 전환을 유도하는 항목에 대한 추천을 강화합니다. 이렇게 하면 카탈로그가 빠르게 변경되거나 뉴스 기사 또는 프로모션과 같은 새 항목이 새것인 상태일 때 사용자에게 더 연관성이 높은 경우, 항목 검색률과 참여율이 향상됩니다.

탐색할 양(상호 작용 데이터 또는 연관성이 적은 항목이 더 자주 추천되는 경우)과 활용할 양(추천이 우리가 알고 있는 내용 또는 연관성을 기반으로 하는 경우)을 균형 있게 조정할 수 있습니다. Amazon Personalize는 암시적인 사용자 피드백에 따라 향후 추천을 자동으로 조정합니다.

먼저 위의 레시피 목록에서 ARN을 찾아 레시피를 선택합니다.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### 솔루션 생성 및 기다리기

먼저 API로 솔루션을 생성한 다음 버전을 생성합니다. 모델을 훈련하고 해당 버전의 솔루션을 만드는 데 몇 분 정도 걸립니다. 이 프로세스가 시작되고 진행률 알림이 표시되면, 잠시 쉬거나 커피를 마시는 등의 시간으로 활용할 수 있습니다.

이 함수는 목표 민감도 설정(OFF, LOW, MED 또는 HIGH)을 받습니다. 이는 모델에서 목표가 미치는 영향을 결정하는 가중치가 조정됩니다.

이 함수는 솔루션을 생성하고 해당 솔루션의 초기 솔루션 버전을 생성합니다.

#### 솔루션 생성

In [ ]:
def create_solution(name, objectiveSensitivity):
    create_solution_response = personalize.create_solution(
        name = name,
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn,
        solutionConfig = {
            "optimizationObjective": {
                "itemAttribute": "ROYALTY",
                "objectiveSensitivity":objectiveSensitivity
            }
        }
    )
    
    solution_arn = create_solution_response['solutionArn']
    
    print('solutionArn:' + solution_arn)

    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print('solution_version_arn: ' + solution_version_arn)

    return {
        "solution_arn": solution_arn,
        "solution_version_arn": solution_version_arn
    }
    
def waitForSolutionVersion(solution_version_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_solution_version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = describe_solution_version_response["solutionVersion"]["status"]
        print("SolutionVersion: {} {}".format(solution_version_arn, status))

        if status == "ACTIVE" or status == "CREATE FAILED"  or status == "CREATE STOPPING":
            break

        time.sleep(60)

#### 솔루션 버전 생성
목표 최적화가 비활성화된 솔루션, LOW로 설정된 솔루션, HIGH로 설정된 솔루션 등 3가지 솔루션을 만듭니다. 이렇게 하면 세 가지 솔루션이 동시에 시작되며, 다음 단계로 솔루션이 완료될 때까지 기다립니다.

In [ ]:
high_solution = create_solution('movie-recommendation-low-royalties-'+suffix, 'HIGH')
low_solution = create_solution('movie-recommendation-medium-royalties-'+suffix, 'LOW')
no_objective_optimization_solution = create_solution('movie-recommendation-max-relevance-'+suffix, 'OFF')

#### 솔루션 버전이 활성 상태가 될 때까지 기다리기

이 작업에는 40~50분 정도 걸립니다.

In [ ]:
waitForSolutionVersion(low_solution['solution_version_arn'])
waitForSolutionVersion(no_objective_optimization_solution['solution_version_arn'])
waitForSolutionVersion(high_solution['solution_version_arn'])

#### 솔루션 버전의 지표 가져오기

이제 솔루션과 버전이 존재하므로 성능을 판단할 지표를 얻을 수 있습니다. 이러한 지표는 데모 데이터 세트이므로 특별히 좋은 것은 아니지만, 데이터 세트가 더 크고 복잡해질수록 향상됩니다.

목표 최적화의 영향을 기준으로 모델의 품질 차이를 확인할 수 있습니다.

In [ ]:
def get_solution_metrics(solutions):
    metricdata = { "name": []}
    
    for key in solutions:
        solution = solutions[key]
        
        metricdata["name"].append(key)
        
        get_solution_metrics_response = personalize.get_solution_metrics(
            solutionVersionArn = solution['solution_version_arn']
        )

        for metricname in get_solution_metrics_response['metrics']:
            if not metricname in metricdata:
                metricdata[metricname] = []
                
            metricdata[metricname].append( get_solution_metrics_response['metrics'][metricname])
            
        # print(json.dumps(get_solution_metrics_response, indent=2))
    return pd.DataFrame.from_dict(metricdata);

metrics = get_solution_metrics({
    "no-optimization": no_objective_optimization_solution,
    "low-optimization": low_solution,
    "high-optimization": high_solution,
})

metrics

지표를 이해하려면 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html)를 읽는 것이 좋지만, 편의를 위해 아래에 설명서의 일부를 복사해 두었습니다.

Personalize에서의 평가와 관련하여 다음 용어를 이해해야 합니다.

- 연관 추천은 특정 사용자에 대한 테스트 데이터의 값과 일치하는 추천을 말합니다.
- 순위는 추천 목록에서 추천 항목의 위치를 나타냅니다. 위치 1(목록 맨 위)은 사용자와 가장 연관성이 높은 것으로 간주됩니다.
- 쿼리는 GetRecommendations 호출에 해당하는 내부 정보를 말합니다.

Personalize에서 생성되는 지표는 다음과 같습니다.

- coverage: 훈련 데이터의 전체 고유 항목 수 중 모든 쿼리의 고유 추천 항목 비율(항목 및 상호 작용 데이터 세트 모두 포함)입니다.
- mean_reciprocal_rank_at_25: 모든 쿼리에 대한 상위 25개 추천 항목 중 첫 번째 관련 추천의 [평균 상호 순위](https://en.wikipedia.org/wiki/Mean_reciprocal_rank)입니다. 이 지표는 순위가 가장 높은 단일 추천 항목을 파악하려는 경우 적합합니다.
- normalized_discounted_cumulative_gain_at_K: 절감 이득은 추천 목록에서 순위가 낮은 추천이 순위가 높은 추천보다 연관성이 적다고 가정합니다. 따라서 각 추천은 해당 위치에 따라 인수에 의해 절감됩니다(낮은 가중치가 부여됨). K에서 [누적 절감 이득](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)(DCG)을 산출하기 위해 상위 K개 추천의 각 연관 절감 추천을 함께 집계합니다. 정규화된 절감 누적 이득(NDCG)은 DCG를 이상적인 DCG로 나누어 NDCG가 0~1이 되도록 합니다. (이상적인 DCG란 상위 K 추천이 연관성에 따라 정렬하는 경우입니다.) Amazon Personalize는 1/log(1 + 위치)라는 가중치 인수를 사용합니다. 여기서 목록의 최상위는 위치 1입니다. 목록의 최상위에 있는 항목이 더 많은 주목을 받게 되므로, 이 지표는 목록 맨 위에 있는 연관 항목에 가중치를 부여합니다.
- precision_at_K: 상위 K개의 추천 중 연관 추천 수를 K로 나눈 값입니다. 이 지표는 연관 항목의 정확한 추천에 대해 가중치를 부여합니다.

## 캠페인 생성 및 기다리기

이제 잘 작동하는 솔루션 버전이 있으므로 애플리케이션에 사용할 캠페인을 만들어야 합니다. 캠페인은 호스팅된 솔루션 버전으로, 추천을 쿼리할 수 있는 엔드포인트입니다. 요금은 처리 용량(초당 사용자의 개인화 요청)을 추정하여 정해집니다. 캠페인을 배포할 때 최소 초당 트랜잭션(TPS) 값(`minProvisionedTPS`)을 설정합니다. 이 서비스는 AWS 내의 다른 서비스와 마찬가지로 수요에 따라 자동으로 확장되지만, 대기 시간이 중요한 경우에는 더 많은 수요에 대비해 미리 프로비저닝하는 것이 좋습니다. 이 데모에서는 최소 처리량 임계값을 1로 설정합니다. 자세한 내용은 [요금](https://aws.amazon.com/personalize/pricing/) 페이지를 참조하세요.

앞서 언급했듯이, 이 솔루션에 사용되는 사용자 개인화 레시피는 "콜드" 항목의 자동 탐색을 지원합니다. 캠페인을 만들 때 수행하는 탐색량을 제어할 수 있습니다. `itemExplorationConfig` 데이터 유형은 `explorationWeight` 및 `explorationItemAgeCutOff` 파라미터를 지원합니다. 탐색 가중치는 상호 작용 데이터 또는 연관성이 적은 항목을 추천에 얼마나 자주 포함할지를 결정합니다. 값이 1.0에 가까울수록 탐색량이 늘어납니다. 0인 경우 탐색이 발생하지 않으며 추천이 현재 데이터(연관성)를 기반으로 합니다. 탐색 항목 보존 기간 컷오프는 최근 상호 작용 이후 기간을 기준으로 탐색할 항목을 결정합니다. 항목 탐색 범위를 정의하기 위해 최근 상호 작용 이후 최대 항목 보존 기간(일)을 제공합니다. 이 값이 클수록 탐색 시에 더 많은 항목이 고려됩니다. 아래의 캠페인에서는 탐색 가중치를 0.5로 지정합니다.

#### 캠페인 생성

In [ ]:
def create_campaign(solution, name):
    create_campaign_response = personalize.create_campaign(
        name = "personalize-demo-" + name + '-' + suffix,
        solutionVersionArn = solution['solution_version_arn'],
        minProvisionedTPS = 1,
        campaignConfig = {
            "itemExplorationConfig": {
                "explorationWeight": "0.5"
            }
        }
    )

    campaign_arn = create_campaign_response['campaignArn']
    print('campaign_arn:' + campaign_arn)
    return campaign_arn

def waitForCampaign(solution):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = solution['campaign_arn']
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {} {}".format(solution['campaign_arn'], status))

        if status == "ACTIVE" or status == "CREATE FAILED":
            break

        time.sleep(60)

#### 3개의 캠페인 생성
각 목표 최적화별로 캠페인을 만듭니다. 단, 다른 모든 설정은 동일하게 유지하여 목표 최적화의 영향을 입증합니다.

In [ ]:
high_solution['campaign_arn'] = create_campaign(high_solution, 'high')
low_solution['campaign_arn'] = create_campaign(low_solution, 'low')
no_objective_optimization_solution['campaign_arn'] = create_campaign(no_objective_optimization_solution, 'max_relevance')


#### 캠페인이 활성 상태가 될 때까지 기다리기

활성 상태가 되는 데 10분 정도 걸립니다.

In [ ]:
waitForCampaign(high_solution)
waitForCampaign(low_solution)
waitForCampaign(no_objective_optimization_solution)

## 샘플 추천 받기

캠페인이 활성 상태가 되면 추천을 받을 준비가 된 것입니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음 ID뿐만 아니라, 추천을 위한 다른 영화 정보도 얻을 수 있도록 몇 가지 헬퍼 함수를 만듭니다.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE'] + '(' + f'{-1*items.iloc[movie_id]["ROYALTY"]:.2f}'+ ')'

def get_movie_royalty(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return -1*items.iloc[movie_id]["ROYALTY"]

#### GetRecommendations 호출

위에서 얻은 사용자를 사용하여 아래의 코드 줄에서 추천을 받고 추천 영화 목록을 반환합니다.


In [ ]:
def get_recommendations(solution):
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = solution['campaign_arn'],
        userId = str(user_id),
    )
    # Update DF rendering
    pd.set_option('display.max_rows', 30)

    item_list = get_recommendations_response['itemList']

    recommendation_list = []

    total_royalties = 0.0
    
    for item in item_list:
        title = get_movie_title(item['itemId'])
        total_royalties = total_royalties + get_movie_royalty(item['itemId'])
        recommendation_list.append(title)
        
    recommendation_list.append('TOTAL ROYALTIES: '+ f'{total_royalties:.2f}')
    return recommendation_list

#### 추천 비교
동일한 사용자에 대한 추천 세트를 만들어 목표 최적화의 영향을 비교합니다.

제목과 연도 뒤에 괄호 안에 표시된 로열티 값의 영향을 주목하세요. 목표 최적화가 비활성화된, 평점과 로열티가 높은 영화는 목표 최적화가 활성화되어 있을 때 목록에서 더 낮은 위치에 표시되는 경향이 있습니다.

또한 각 추천 세트에 있는 모든 타이틀의 총 로열티도 주목하세요.

In [ ]:
recommendations_df = pd.DataFrame(get_recommendations(no_objective_optimization_solution), columns = ['ObjectiveOff'])
recommendations_df['LowObjective'] = get_recommendations(low_solution)
recommendations_df['HighObjective'] = get_recommendations(high_solution)
recommendations_df

## 검토

이 노트북에서는 특정 타이틀에 대해 지불해야 할 로열티를 고려한 영화 추천 엔진의 예를 단계별로 살펴보았습니다. 이 가중치를 추천 알고리즘에 포함함으로써, 스트리밍 서비스는 사용자에게 양질의 추천을 제공할 뿐만 아니라 콘텐츠 제작자에게 지불해야 할 로열티를 최소화할 수 있습니다.

위의 차트에서 목표 최적화가 비활성화된 경우, 로열티 수수료가 고려되지 않기 때문에 예상대로 수수료가 상당히 균일하게 분포합니다. 하지만 LowObjective 열에서는 값이 더 낮고 로열티 합계가 가장 높은 목표 최적화 설정에 대해 가장 낮습니다.

## 정리

리소스 정리

In [ ]:
def delete_campaign(campaign_arn):
    delete_campaign_result = personalize.delete_campaign(campaignArn=campaign_arn )
    

def wait_for_delete_campaign(campaign_arn):
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_campaign_response = personalize.describe_campaign(
                campaignArn = campaign_arn
            )
            status = describe_campaign_response["campaign"]["status"]
            print("campaign: {}".format(status))

        except ClientError as e:
            print(e)
            break

        time.sleep(10)
    print('campaign ' + campaign_arn + ' deleted')
    
def delete_solution(solution_arn):
    delete_solution_result = personalize.delete_solution(solutionArn=solution_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours

def wait_for_delete_solution(solution_arn):
    while time.time() < max_time:
        
        try:
            describe_solution_response = personalize.describe_solution(
                solutionArn = solution_arn
            )
            status = describe_solution_response["solution"]["status"]
            print("Solution: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('Solution ' + solution_arn + ' deleted')
    
def delete_dataset(dataset_arn):
    delete_dataset_result = personalize.delete_dataset(datasetArn=dataset_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_response = personalize.describe_dataset(
                datasetArn = dataset_arn
            )
            status = describe_dataset_response["dataset"]["status"]
            print("dataset: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset ' + dataset_arn + ' deleted')
    
def delete_schema(schema_arn):
    delete_schema_result = personalize.delete_schema(schemaArn=schema_arn )
    

    print('schema ' + schema_arn + ' deleted')
    
def delete_dataset_group(dataset_group_arn):
    delete_dataset_group_result = personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn )
    
    max_time = time.time() + 3*60*60 # 3 hours
    while time.time() < max_time:
        try:
            describe_dataset_group_response = personalize.describe_dataset_group(
                datasetGroupArn = dataset_group_arn
            )
            status = describe_dataset_group_response["datasetGroup"]["status"]
            print("dataset_group: {}".format(status))

        except ClientError:
            break
        time.sleep(10)
    print('dataset_group ' + dataset_group_arn + ' deleted')
    
def delete_all(solutions):
    for solution in solutions:
        delete_campaign(solution['campaign_arn'])
        
    for solution in solutions:
        wait_for_delete_campaign(solution['campaign_arn'])
    for solution in solutions:
        delete_solution(solution['solution_arn'])
    for solution in solutions:
        wait_for_delete_solution(solution['solution_arn'])


    

In [ ]:
delete_all([no_objective_optimization_solution, low_solution, high_solution] )

In [ ]:
delete_dataset(item_dataset_arn)
delete_dataset(interaction_dataset_arn)

In [ ]:
delete_schema(item_schema_arn)
delete_schema(schema_arn)

In [ ]:
delete_dataset_group(dataset_group_arn)

In [ ]:
iam = boto3.client("iam")

iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess')
iam.detach_role_policy(RoleName=role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess')
time.sleep(10) # propogation time

iam.delete_role(RoleName=role_name)

In [ ]:
! aws s3 rm --recursive s3://$bucket_name

In [ ]:
s3.delete_bucket(Bucket=bucket_name)